In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import json
import numpy as np
from scripts.load_data import *

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor

In [3]:
configuration = load_configuration("../configuration/configuration.json")

paths = configuration["paths"]
data_student_numbers_f = pd.read_excel(paths["path_student_count_first-years"])
data_student_numbers_h = pd.read_excel(paths["path_student_count_higher-years"])
data_student_numbers_v = pd.read_excel(paths["path_student_volume"])

In [4]:
combined_data_student_numbers = data_student_numbers_f.merge(data_student_numbers_h, on=["Croho groepeernaam", "Herkomst", "Collegejaar"])
combined_data_student_numbers = combined_data_student_numbers.rename(columns={"Aantal_studenten_x": "Aantal_studenten_f", "Aantal_studenten_y": "Aantal_studenten_h"})

In [5]:
starting_year = 2021
ending_year = 2022
predict_year = 2023

In [6]:
ratio_data = combined_data_student_numbers[(combined_data_student_numbers["Collegejaar"] >= starting_year) & (combined_data_student_numbers["Collegejaar"] <= ending_year)]
ratio_data["Ratio"] = ratio_data["Aantal_studenten_h"].divide(ratio_data["Aantal_studenten_f"])
ratio_data["Ratio"] = ratio_data['Ratio'].replace(np.inf, np.nan)
ratio_data[ratio_data["Croho groepeernaam"] == "M Geneeskunde"]

ratio = ratio_data[["Croho groepeernaam", "Herkomst", "Ratio"]].groupby(['Croho groepeernaam', 'Herkomst'], as_index=False).mean()

C:\Users\jjble\AppData\Local\Temp\ipykernel_9888\1705177450.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratio_data["Ratio"] = ratio_data["Aantal_studenten_h"].divide(ratio_data["Aantal_studenten_f"])
C:\Users\jjble\AppData\Local\Temp\ipykernel_9888\1705177450.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratio_data["Ratio"] = ratio_data['Ratio'].replace(np.inf, np.nan)


In [7]:
data = combined_data_student_numbers
train = data[(data["Collegejaar"] < predict_year)]
test = data[(data["Collegejaar"] == predict_year)]

train = train.drop_duplicates()

X_train = train.drop(['Aantal_studenten_h'], axis=1)
y_train = train.pop('Aantal_studenten_h')

test = test.sort_values(by=["Croho groepeernaam", "Herkomst"])

numeric_cols = ['Collegejaar', 'Aantal_studenten_f']
categorical_cols = ['Croho groepeernaam', 'Herkomst']

numeric_transformer = "passthrough"  # No transformation for numeric columns
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_cols),
        ('categorical', categorical_transformer, categorical_cols)
    ])

X_train = preprocessor.fit_transform(X_train)
preprocessed_test = preprocessor.transform(test)

model = XGBRegressor(learning_rate=0.5301857237491415)

model.fit(X_train, y_train)

predictie = model.predict(preprocessed_test)

In [21]:
total_mae_ratio = 0.0
total_mape_ratio = 0.0

total_mae_xgboost = 0.0
total_mape_xgboost = 0.0

total_mae_ensemble = 0.0
total_mape_ensemble = 0.0

i = 0
count = 0.0

for _, row in test.iterrows():
    programme = row["Croho groepeernaam"]
    origin = row["Herkomst"]
    first_year_row = data_student_numbers_f[(data_student_numbers_f["Croho groepeernaam"] == programme) &
                                            (data_student_numbers_f["Herkomst"] == origin) &
                                            (data_student_numbers_f["Collegejaar"] == predict_year)]
    higher_year_row = data_student_numbers_h[(data_student_numbers_h["Croho groepeernaam"] == programme) & 
                                             (data_student_numbers_h["Herkomst"] == origin) &
                                             (data_student_numbers_h["Collegejaar"] == predict_year)]
    current_ratio = ratio[(ratio["Croho groepeernaam"] == programme) & 
                          (ratio["Herkomst"] == origin)]

    if len(first_year_row) > 0 and len(higher_year_row) > 0 and len(current_ratio) > 0:
        true_label = higher_year_row["Aantal_studenten"].iloc[0]
        ratio_pred = first_year_row["Aantal_studenten"].iloc[0] * current_ratio["Ratio"].iloc[0]
        xgboost_pred = predictie[i]

        mae_ratio = abs(true_label - ratio_pred)
        mape_ratio = abs((true_label - ratio_pred) / true_label)

        mae_xgboost = abs(true_label - xgboost_pred)
        mape_xgboost = abs((true_label - xgboost_pred) / true_label)

        ensemble_prediction = ratio_pred * 0.16 + xgboost_pred * 0.84
        mae_ensemble = abs(true_label - ensemble_prediction)
        mape_ensemble = abs((true_label - ensemble_prediction) / true_label)
        
        # print(f"{programme}, {origin}: {mae_ratio} & {mape} => {predictie[i]} & {higher_year_row['Aantal_studenten'].iloc[0]}")

        total_mae_ratio += mae_ratio
        total_mape_ratio += mape_ratio
        
        total_mae_xgboost += mae_xgboost
        total_mape_xgboost += mape_xgboost

        total_mae_ensemble += mae_ensemble
        total_mape_ensemble += mape_ensemble

        count += 1.0
        
    i += 1

print(f"Final MAE: ratio {total_mae_ratio / count} & xgboost {total_mae_xgboost / count}")
print(f"Final MAPE: ratio {total_mape_ratio / count} & xgboost {total_mape_xgboost / count}")
print(f"Final ensemble: MAE {total_mae_ensemble / count} & MAPE {total_mape_ensemble / count}")
print(f"Count: {i}")

# Fifty-fifty:
# Final MAE: ratio 21.045883668539968 & xgboost 15.430184893092218
# Final MAPE: ratio 0.5404465423649051 & xgboost 0.5144048317787553
# Final ensemble: MAE 16.982791701778673 & MAPE 0.5042866041640258
# Count: 184

Final MAE: ratio 21.045883668539968 & xgboost 15.430184893092218
Final MAPE: ratio 0.5404465423649051 & xgboost 0.5144048317787553
Final ensemble: MAE 15.331977989398965 & MAPE 0.5026494514340419
Count: 184


In [19]:
import optuna

def objective(trial):
    ensemble_splitter = trial.suggest_float("ensemble_splitter", 0.0, 1.0)

    total_mae_ratio = 0.0
    total_mape_ratio = 0.0

    total_mae_xgboost = 0.0
    total_mape_xgboost = 0.0

    total_mae_ensemble = 0.0
    total_mape_ensemble = 0.0

    i = 0
    count = 0.0

    for _, row in test.iterrows():
        programme = row["Croho groepeernaam"]
        origin = row["Herkomst"]
        first_year_row = data_student_numbers_f[(data_student_numbers_f["Croho groepeernaam"] == programme) &
                                                (data_student_numbers_f["Herkomst"] == origin) &
                                                (data_student_numbers_f["Collegejaar"] == predict_year)]
        higher_year_row = data_student_numbers_h[(data_student_numbers_h["Croho groepeernaam"] == programme) & 
                                                (data_student_numbers_h["Herkomst"] == origin) &
                                                (data_student_numbers_h["Collegejaar"] == predict_year)]
        current_ratio = ratio[(ratio["Croho groepeernaam"] == programme) & 
                            (ratio["Herkomst"] == origin)]

        if len(first_year_row) > 0 and len(higher_year_row) > 0 and len(current_ratio) > 0:
            true_label = higher_year_row["Aantal_studenten"].iloc[0]
            ratio_pred = first_year_row["Aantal_studenten"].iloc[0] * current_ratio["Ratio"].iloc[0]
            xgboost_pred = predictie[i]

            mae_ratio = abs(true_label - ratio_pred)
            mape_ratio = abs((true_label - ratio_pred) / true_label)

            mae_xgboost = abs(true_label - xgboost_pred)
            mape_xgboost = abs((true_label - xgboost_pred) / true_label)

            ensemble_prediction = ratio_pred * ensemble_splitter + xgboost_pred * (1.0 - ensemble_splitter)
            mae_ensemble = abs(true_label - ensemble_prediction)
            mape_ensemble = abs((true_label - ensemble_prediction) / true_label)
            
            # print(f"{programme}, {origin}: {mae_ratio} & {mape} => {predictie[i]} & {higher_year_row['Aantal_studenten'].iloc[0]}")

            total_mae_ratio += mae_ratio
            total_mape_ratio += mape_ratio
            
            total_mae_xgboost += mae_xgboost
            total_mape_xgboost += mape_xgboost

            total_mae_ensemble += mae_ensemble
            total_mape_ensemble += mape_ensemble

            count += 1.0
            
        i += 1

    print(f"Final MAE: ratio {total_mae_ratio / count} & xgboost {total_mae_xgboost / count}")
    print(f"Final MAPE: ratio {total_mape_ratio / count} & xgboost {total_mape_xgboost / count}")
    print(f"Final ensemble: MAE {total_mae_ensemble / count} & MAPE {total_mape_ensemble / count}")

    return np.mean([total_mae_ensemble / count, total_mape_ensemble / count])

In [20]:
study = optuna.create_study()
study.optimize(objective, n_trials=200)

print(study.best_params)

[I 2024-05-14 15:59:45,337] A new study created in memory with name: no-name-2ec59b72-3593-47bb-bc42-cec544ea57f9
[I 2024-05-14 15:59:46,132] Trial 0 finished with value: 8.049827807714268 and parameters: {'ensemble_splitter': 0.29864118927856087}. Best is trial 0 with value: 8.049827807714268.


Final MAE: ratio 21.045883668539968 & xgboost 15.430184893092218
Final MAPE: ratio 0.5404465423649051 & xgboost 0.5144048317787553
Final ensemble: MAE 15.599799175785174 & MAPE 0.49985643964336224


[I 2024-05-14 15:59:46,802] Trial 1 finished with value: 7.931271206439595 and parameters: {'ensemble_splitter': 0.08291086595009756}. Best is trial 1 with value: 7.931271206439595.


Final MAE: ratio 21.045883668539968 & xgboost 15.430184893092218
Final MAPE: ratio 0.5404465423649051 & xgboost 0.5144048317787553
Final ensemble: MAE 15.355594455395014 & MAPE 0.506947957484176


[I 2024-05-14 15:59:47,415] Trial 2 finished with value: 9.179474140561284 and parameters: {'ensemble_splitter': 0.6112260317441568}. Best is trial 1 with value: 7.931271206439595.


...

Final MAE: ratio 21.045883668539968 & xgboost 15.430184893092218
Final MAPE: ratio 0.5404465423649051 & xgboost 0.5144048317787553
Final ensemble: MAE 15.355817467045075 & MAPE 0.5013819350683472


[I 2024-05-14 16:02:00,847] Trial 198 finished with value: 7.919336790041842 and parameters: {'ensemble_splitter': 0.13910882530277024}. Best is trial 131 with value: 7.917261114396344.


Final MAE: ratio 21.045883668539968 & xgboost 15.430184893092218
Final MAPE: ratio 0.5404465423649051 & xgboost 0.5144048317787553
Final ensemble: MAE 15.335093383660329 & MAPE 0.5035801964233559


[I 2024-05-14 16:02:01,950] Trial 199 finished with value: 7.917270560998305 and parameters: {'ensemble_splitter': 0.16196742392050362}. Best is trial 131 with value: 7.917261114396344.


Final MAE: ratio 21.045883668539968 & xgboost 15.430184893092218
Final MAPE: ratio 0.5404465423649051 & xgboost 0.5144048317787553
Final ensemble: MAE 15.331973907892955 & MAPE 0.502567214103654
{'ensemble_splitter': 0.16120402211734286}
